In [18]:
import requests as r
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3 as sql
import numpy as np
import datetime as dt
from sqlalchemy import create_engine
from sqlalchemy import text


In [19]:
def log_progress(stage):
    with open("code_log.txt", "a") as log_file:
        log_file.write(f"Progress at stage: {stage}\n")

In [20]:
def extract(url, table_attribs):
    log_progress("Starting extraction")
    
    # Send a GET request to the URL
    response = r.get(url)
    log_progress("Received response from URL")
    
    # Parse the HTML content using BeautifulSoup
    soup = BeautifulSoup(response.content, 'html.parser')
    log_progress("Parsed HTML content")
    
    # Find the table with the specified attributes
    table = soup.find('table', attrs=table_attribs)
    log_progress("Found the table in HTML content")
    
    # Extract the table headers
    headers = [header.text.strip() for header in table.find_all('th')]
    
    # Extract the table rows
    rows = []
    for row in table.find_all('tr')[1:]:  # Skip the header row
        cells = row.find_all('td')
        row_data = [cell.text.strip() for cell in cells]
        rows.append(row_data)
    
    # Create a DataFrame from the extracted data
    df = pd.DataFrame(rows, columns=headers)
    log_progress("Created DataFrame from extracted data")
    
    log_progress("Extraction completed")
    return df


def transform(df, exchange_rates_path):

    log_progress("Starting transformation")
    
    # Load exchange rates from CSV file
    exchange_rates = pd.read_csv(exchange_rates_path)
    
    # Extract exchange rates
    usd_to_gbp = exchange_rates.loc[exchange_rates['Currency'] == 'GBP', 'Rate'].values[0]
    usd_to_eur = exchange_rates.loc[exchange_rates['Currency'] == 'EUR', 'Rate'].values[0]
    usd_to_inr = exchange_rates.loc[exchange_rates['Currency'] == 'INR', 'Rate'].values[0]
    
    # Check if 'Market cap(US$ billion)' column exists
    if 'Market cap(US$ billion)' in df.columns:
        # Convert Market Capitalization to float
        df['Market cap(US$ billion)'] = df['Market cap(US$ billion)'].replace('[\$,]', '', regex=True).astype(float)
        
        # Add new columns for Market Capitalization in GBP, EUR, and INR
        df['Market Capitalization (GBP)'] = (df['Market cap(US$ billion)'] * usd_to_gbp).round(2)
        df['Market Capitalization (EUR)'] = (df['Market cap(US$ billion)'] * usd_to_eur).round(2)
        df['Market Capitalization (INR)'] = (df['Market cap(US$ billion)'] * usd_to_inr).round(2)
    else:
        log_progress("'Market cap(US$ billion)' column not found in DataFrame")
    
    log_progress("Transformation completed")
    return df

def load_to_csv(df, csv_path):

    log_progress("Starting CSV load")
    df.to_csv(csv_path, index=False)
    log_progress("CSV load completed")

def load_to_db(df, sql_connection, table_name):

    log_progress("Starting DB load")
    engine = create_engine(sql_connection)
    df.to_sql(table_name, engine, if_exists='replace', index=False)
    log_progress("DB load completed")
    
def run_query(query_statement, sql_connection):

    log_progress("Starting query execution")
    engine = create_engine(sql_connection)
    with engine.connect() as connection:
        result = connection.execute(text(query_statement))
        for row in result:
            print(row)
    log_progress("Query execution completed")
    
def verify_log_entries(log_file_path):

    with open(log_file_path, 'r') as log_file:
        log_contents = log_file.read()
        print(log_contents)


<>:49: SyntaxWarning: invalid escape sequence '\$'
<>:49: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_1255/1679863111.py:49: SyntaxWarning: invalid escape sequence '\$'
  df['Market cap(US$ billion)'] = df['Market cap(US$ billion)'].replace('[\$,]', '', regex=True).astype(float)


In [21]:
url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = {'class': 'wikitable'}  
exchange_rates_path = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-PY0221EN-Coursera/labs/v2/exchange_rate.csv'  # Path to the exchange rates CSV file
csv_path = 'Largest_banks_data.csv'
sql_connection = 'sqlite:///Banks.db'  
table_name = 'Largest_banks'
query_statement = 'SELECT * FROM Largest_banks'
log_file_path = 'code_log.txt'

In [22]:
try:
	# Extract data
	df = extract(url, table_attribs)

	# Transform data
	df_transformed = transform(df, exchange_rates_path)

	# Load to CSV
	load_to_csv(df_transformed, csv_path)

	# Load to DB
	load_to_db(df_transformed, sql_connection, table_name)

	# Run query
	run_query(query_statement, sql_connection)

	# Verify log entries
	verify_log_entries(log_file_path)
except Exception as e:
	log_progress(f"Error occurred: {e}")
	print(f"Error occurred: {e}")

('1', 'JPMorgan Chase', 432.92, 346.34, 402.62, 35910.71)
('2', 'Bank of America', 231.52, 185.22, 215.31, 19204.58)
('3', 'Industrial and Commercial Bank of China', 194.56, 155.65, 180.94, 16138.75)
('4', 'Agricultural Bank of China', 160.68, 128.54, 149.43, 13328.41)
('5', 'HDFC Bank', 157.91, 126.33, 146.86, 13098.63)
('6', 'Wells Fargo', 155.87, 124.7, 144.96, 12929.42)
('7', 'HSBC Holdings PLC', 148.9, 119.12, 138.48, 12351.26)
('8', 'Morgan Stanley', 140.83, 112.66, 130.97, 11681.85)
('9', 'China Construction Bank', 139.82, 111.86, 130.03, 11598.07)
('10', 'Bank of China', 136.81, 109.45, 127.23, 11348.39)
Progress at stage: Starting extraction
Progress at stage: Received response from URL
Progress at stage: Parsed HTML content
Progress at stage: Found the table in HTML content
Progress at stage: Created DataFrame from extracted data
Progress at stage: Extraction completed
Progress at stage: Starting transformation
Progress at stage: Starting extraction
Progress at stage: Receive